# Running a CAT based on a synthetic correct/incorrect 3PL IRT model

This example shows how to run a CAT based on a synthetic correct/incorrect 3PL
IRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: AbilityVarianceStateCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
using ComputerAdaptiveTesting.Responses: BooleanResponse
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
using ComputerAdaptiveTesting.DummyData: dummy_full, std_normal, SimpleItemBankSpec, StdModel3PL
using ComputerAdaptiveTesting.MathTraits
(item_bank, question_labels, abilities, responses) = dummy_full(
    Random.default_rng(42),
    SimpleItemBankSpec(StdModel3PL(), OneDimContinuousDomain(), BooleanResponse());
    num_questions=100,
    num_testees=3
)

(ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}}([0.05009926153760216, 0.006842482995374766, 0.1804990103143584, 0.0, 0.04606303607559311, 0.0, 0.0, 0.0, 0.3140655043499787, 0.0  …  0.0, 0.10050581117854676, 0.0, 0.0, 0.0, 0.03829020892488844, 0.19655905333711238, 0.2756935023062041, 0.0, 0.0], ComputerAdaptiveTesting.ItemBanks.TransferItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [1.6110433255354544, -1.121680473800278, 0.43651385325650716, 0.9596241951011495, -0.4795390437119982, -0.29978586212162817, -0.7615020048370765, -1.7906555509685327, 0.7896019130980401, 1.0058938488914813  …  -0.4547683032982378, -0.06934738536159746, 0.2329267602546866, -0.03479294424389485, -0.46397515053149113, 2.088858763937631, -1.4475885507080912, 0.19538906503540215, -0.23532747757927455, -2.5327688266982875], [1.0774234860042422, 1.15380968954479, 0.9286612710

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [3]:
max_questions = 99
integrator = FixedGKIntegrator(-6, 6, 80)
dist_ability_est = PriorAbilityEstimator(std_normal)
ability_estimator = MeanAbilityEstimator(dist_ability_est, integrator)
rules = CatRules(
    ability_estimator,
    AbilityVarianceStateCriterion(dist_ability_est, integrator),
    FixedItemsTerminationCondition(max_questions)
)

points = 500
xs = range(-2.5, 2.5, length=points)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator)
for testee_idx in axes(responses, 2)
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[testee_idx]
    abs_err = abs(θ - true_θ)
end

Make a plot showing how the estimated value evolves during the CAT.
We also plot the 'true' values used to generate the responses.

In [4]:
conv_lines_fig = ability_evolution_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

Make an interactive plot, showing how the distribution of the ability
likelihood evolves.

In [5]:
conv_dist_fig = lh_evolution_interactive(recorder; abilities=abilities)
conv_dist_fig

Figure()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*